In [2]:
 import pandas as pd

In [3]:
data = pd.read_csv('/content/drive/MyDrive/datasets/xai/hw4/data.csv')
data.head()

,age,fnlwgt,educational-num,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,226802,7,0,1,0,0,40,0,0,...,False,False,False,False,False,False,False,True,False,False
1,38,89814,9,1,1,0,0,50,0,0,...,False,False,False,False,True,False,False,False,False,False
2,28,336951,12,1,1,0,0,40,0,1,...,True,False,False,False,True,False,False,False,False,False
3,44,160323,10,0,1,7688,0,40,0,1,...,False,False,False,False,True,False,False,False,False,False
4,18,103497,10,1,0,0,0,30,0,0,...,False,False,False,False,False,False,False,True,False,False


In [4]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

### metrics

In [5]:
import numpy as np
import pandas as pd

def calculate_fairness_metrics(df, y_pred, sensitive_attr_column, positive_class=1):

    assert len(df) == len(y_pred), "The length of y_pred must match the number of rows in the DataFrame."

    df['prediction'] = y_pred

    groups = df[sensitive_attr_column].unique()
    if len(groups) != 2:
        raise ValueError("The sensitive attribute column must contain exactly two unique groups.")

    group1, group2 = groups

    prob_C1_G1 = df[(df['prediction'] == positive_class) & (df[sensitive_attr_column] == group1)].shape[0] / df[df[sensitive_attr_column] == group1].shape[0]
    prob_C1_G2 = df[(df['prediction'] == positive_class) & (df[sensitive_attr_column] == group2)].shape[0] / df[df[sensitive_attr_column] == group2].shape[0]

    zemel_fairness = prob_C1_G2 - prob_C1_G1

    disparate_impact = prob_C1_G1 / prob_C1_G2

    return zemel_fairness, disparate_impact



### base model

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

X = data.drop(columns=['income'])
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

numerical_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier())
])

param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.2],
}

grid_search = GridSearchCV(model, param_grid, cv=2, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

print("Best parameters found: ", grid_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters found:  {'classifier__learning_rate': 0.2, 'classifier__n_estimators': 200}
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     11233
           1       0.77      0.55      0.64      3420

    accuracy                           0.86     14653
   macro avg       0.82      0.75      0.78     14653
weighted avg       0.85      0.86      0.85     14653



In [20]:

zemel_fairness, disparate_impact = calculate_fairness_metrics(X_test, y_pred, 'gender')

print(f"Zemel Fairness: {zemel_fairness}")
print(f"Disparate Impact: {disparate_impact}")


Zemel Fairness: 0.18976358574691277
Disparate Impact: 0.17183318455421243


### training without gender column

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.drop(columns=['gender'])

categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

numerical_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier())
])

param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.2]
    }

grid_search = GridSearchCV(model, param_grid, cv=2, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

print("Best parameters found: ", grid_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters found:  {'classifier__learning_rate': 0.2, 'classifier__n_estimators': 200}
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     11233
           1       0.77      0.55      0.64      3420

    accuracy                           0.86     14653
   macro avg       0.82      0.75      0.78     14653
weighted avg       0.85      0.86      0.85     14653



In [9]:

zemel_fairness, disparate_impact = calculate_fairness_metrics(X_test, y_pred, 'gender')

print(f"Zemel Fairness: {zemel_fairness}")
print(f"Disparate Impact: {disparate_impact}")


Zemel Fairness: 0.18976358574691277
Disparate Impact: 0.17183318455421243


### Fair model

In [10]:
new_data = data.copy()
X = new_data.drop(columns=['income'])
y = new_data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train['prediction'] = best_model.predict(X_train)
X_train['prediction_prob'] = np.max(best_model.predict_proba(X_train),axis=1)
X_train['income'] = y_train


In [11]:
n_men = X_train[X_train['gender'] == 1].shape[0]
n_women = X_train[X_train['gender'] == 0].shape[0]
n_men_plus = X_train[(X_train['income'] == 1) & (X_train['gender'] == 1)].shape[0]
n_women_plus = X_train[(X_train['income'] == 1) & (X_train['gender'] == 0)].shape[0]

n= int((n_women*n_men_plus-n_men*n_women_plus )/new_data.shape[0])
print(n)

1034


In [12]:
cp_group = X_train[(X_train['income'] == 1) & (X_train['gender'] == 1)]
cd_group = X_train[(X_train['income'] == 0) & (X_train['gender'] == 0)]

In [13]:
cp_group_sorted = cp_group.sort_values(by='prediction_prob', ascending=False)

cd_group_sorted = cd_group.sort_values(by='prediction_prob', ascending=False)

cp_top_n = cp_group_sorted.head(n)
cd_top_n = cd_group_sorted.head(n)

X_train.loc[cp_top_n.index[:n], 'income'] = 0
X_train.loc[cd_top_n.index[:n], 'income'] = 1


In [14]:
y_train = X_train['income']
X_train = X_train.drop(columns=['prediction', 'prediction_prob', 'income'])

In [15]:
X_train.head()

,age,fnlwgt,educational-num,race,gender,capital-gain,capital-loss,hours-per-week,native-country,workclass_Federal-gov,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
42392,25,188767,9,1,1,0,0,40,0,False,...,False,False,False,False,False,False,False,True,False,False
14623,64,286732,9,1,0,0,0,17,0,False,...,False,True,False,False,False,True,False,False,False,False
27411,29,253801,9,1,1,0,0,40,1,False,...,False,False,False,False,True,False,False,False,False,False
1288,28,334032,11,1,1,0,0,50,0,False,...,False,False,False,False,True,False,False,False,False,False
7078,22,173004,9,0,1,0,0,1,0,False,...,False,False,False,False,False,False,True,False,False,False


In [16]:
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

numerical_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier())
])

param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.2],
}

grid_search = GridSearchCV(model, param_grid, cv=2, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

print("Best parameters found: ", grid_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters found:  {'classifier__learning_rate': 0.2, 'classifier__n_estimators': 200}
              precision    recall  f1-score   support

           0       0.84      0.91      0.87     11233
           1       0.58      0.41      0.48      3420

    accuracy                           0.79     14653
   macro avg       0.71      0.66      0.68     14653
weighted avg       0.78      0.79      0.78     14653



In [17]:

zemel_fairness, disparate_impact = calculate_fairness_metrics(X_test, y_pred, 'gender')

print(f"Zemel Fairness: {zemel_fairness}")
print(f"Disparate Impact: {disparate_impact}")


Zemel Fairness: 0.050176235284230736
Disparate Impact: 0.72415734253728


### fairness using another method

In [21]:
y_probs = best_model.predict_proba(X_test)[:, 1]

def predict_with_threshold(probs, threshold):
    return (probs >= threshold).astype(int)

threshold = 0.85


y_pred_custom_threshold = predict_with_threshold(y_probs, threshold)
print(classification_report(y_test, y_pred_custom_threshold))

zemel_fairness, disparate_impact = calculate_fairness_metrics(X_test, y_pred_custom_threshold, 'gender')

print(f"Zemel Fairness: {zemel_fairness}")
print(f"Disparate Impact: {disparate_impact}")

              precision    recall  f1-score   support

           0       0.82      1.00      0.90     11233
           1       0.98      0.29      0.45      3420

    accuracy                           0.83     14653
   macro avg       0.90      0.64      0.67     14653
weighted avg       0.86      0.83      0.79     14653

Zemel Fairness: 0.059493493748893525
Disparate Impact: 0.32348581702244283
